## Sources

In [5]:
import pandas as pd
import numpy as np
import csv
import uuid

In [6]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/gentilly/SOURCES.csv"
OUTPUT_PATH = ROOT + "data/rdf/"

sources = pd.read_csv(PATH,header=0)
display(sources)

,CODE_ARCHIVES,rdf:type,rico:name,rico:location,rico:identifier,rsfs:comment/Folios,hasSourceType,hasCreationDate,hasValidTime/hasBeginning,hasValidTime/hasEnd,rico:isOrWasIncludedIn,rico:isOrWasInstantiationOf,rico:isOrWasDigitalInstantiationOf,rico:hasOrHadDerivedInstanciation,rico:hasOrHadPhysicalLocation
0,94,RecordSet,Matrices des propriétés bâties et non bâties,Gentilly,MAT_B_NB_1813,NaN,MatriceCadastrale1807_1822_Bati_NonBati,1813.0,1813.0,1836.0,CAD1,NaN,NaN,NaN,NaN
1,94,Instanciation,NaN,Gentilly,3P255,NB : 1-542 ; B : 1-166,NaN,NaN,NaN,NaN,NaN,MAT_B_NB_1813,NaN,FRAD094_3P_000255_01,Archives Départementales du Val-de-Marne (94)
2,94,Instanciation,NaN,Gentilly,FRAD094_3P_000255_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_B_NB_1813,NaN,NaN
3,94,RecordSet,Matrice des propriétés foncières (1836-1848),Gentilly,MAT_NB_1836,NaN,MatriceCadastrale1822_1914_NonBati,1836.0,1836.0,1848.0,CAD1,NaN,NaN,NaN,NaN
4,94,Instanciation,NaN,Gentilly,3P256,1-480,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000256_01,Archives Départementales du Val-de-Marne (94)
5,94,Instanciation,NaN,Gentilly,3P257,481-980,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000257_01,Archives Départementales du Val-de-Marne (94)
6,94,Instanciation,NaN,Gentilly,3P258,981-1150,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000258_01,Archives Départementales du Val-de-Marne (94)
7,94,Instanciation,NaN,Gentilly,FRAD094_3P_000256_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN
8,94,Instanciation,NaN,Gentilly,FRAD094_3P_000257_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN
9,94,Instanciation,NaN,Gentilly,FRAD094_3P_000258_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN


In [7]:
# Print the list of column headers
print(sources.columns.tolist())

['CODE_ARCHIVES', 'rdf:type', 'rico:name', 'rico:location', 'rico:identifier', 'rsfs:comment/Folios', 'hasSourceType', 'hasCreationDate', 'hasValidTime/hasBeginning', 'hasValidTime/hasEnd', 'rico:isOrWasIncludedIn', 'rico:isOrWasInstantiationOf', 'rico:isOrWasDigitalInstantiationOf', 'rico:hasOrHadDerivedInstanciation', 'rico:hasOrHadPhysicalLocation']


In [8]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/source/")
g.bind('source', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")
g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

srctype = Namespace("http://data.ign.fr/id/codes/cadastre/sourceType/")
mlclasse= Namespace("http://data.ign.fr/id/codes/cadastre/mlClasse/")
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)

# Iterate over each row in the DataFrame
for index, row in sources.iterrows():

    #URI = http://data.ign.fr/id/sources/{location}_{identifier}
    subject_uri = URIRef(baseuri + f"{str(row['CODE_ARCHIVES'])}_{str(row['rico:location'])}_{str(row['rico:identifier'])}")
    g.add((subject_uri, rico.identifier, Literal(row["rico:identifier"])))

    if row["rdf:type"] == "RecordSet":
        g.add((subject_uri, RDF.type, rico.RecordSet))
        g.add((subject_uri, rico.name, Literal(row["rico:name"])))
        g.add((subject_uri, rico.location, Literal(row["rico:location"])))
        g.add((subject_uri, cad.hasSourceType, URIRef(srctype + f'{row["hasSourceType"]}')))
        if not pd.isnull(row["rico:isOrWasIncludedIn"]):
            g.add((subject_uri, rico.isOrWasIncludedIn, URIRef(baseuri + f"{str(row['CODE_ARCHIVES'])}_{str(row['rico:location'])}_{str(row['rico:isOrWasIncludedIn'])}")))
        creationDateBNode = BNode()
        g.add((subject_uri, rico.hasCreationDate, creationDateBNode))
        g.add((creationDateBNode, RDF.type, add.CrispTimeInstant))
        g.add((creationDateBNode, add.timeCalendar, time.Gregorian))
        g.add((creationDateBNode, add.timePrecision, time.Year))
        str_date = "{:.0f}".format(row["hasCreationDate"])
        g.add((creationDateBNode, add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        validTimeNode = BNode()
        validTimeNodeBegin = BNode()
        validTimeNodeEnd = BNode()
        g.add((subject_uri, add.hasTime, validTimeNode))
        g.add((validTimeNode, RDF.type, time.Interval))
        g.add((validTimeNode, add.hasBeginning, validTimeNodeBegin))
        g.add((validTimeNodeBegin, add.timeCalendar, time.Gregorian))
        g.add((validTimeNodeBegin, add.timePrecision, time.Year))
        str_date = "{:.0f}".format(row["hasValidTime/hasBeginning"])
        g.add((validTimeNodeBegin, add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        g.add((validTimeNode, add.hasEnd, validTimeNodeEnd))
        g.add((validTimeNodeEnd, add.timeCalendar, time.Gregorian))
        g.add((validTimeNodeEnd, add.timePrecision, time.Year))
        str_date = "{:.0f}".format(row["hasValidTime/hasEnd"])
        g.add((validTimeNodeEnd, add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
    elif row["rdf:type"] == "Instanciation":
        if row["rico:identifier"][0:2] == '3P' :
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasInstantiationOf,URIRef(baseuri + f'{str(row["CODE_ARCHIVES"])}_{row["rico:location"]}_{row["rico:isOrWasInstantiationOf"]}')))
            g.add((subject_uri, rico.hasOrHadDerivedInstanciation, URIRef(baseuri + f'{str(row["CODE_ARCHIVES"])}_{row["rico:location"]}_{row["rico:hasOrHadDerivedInstanciation"]}')))
            g.add((subject_uri,rico.hasOrHadPhysicalLocation,Literal(row['rico:hasOrHadPhysicalLocation'])))
        elif row["rico:identifier"][0:2] != '3P' :
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasDigitalInstantiationOf,URIRef(baseuri + f'{str(row["CODE_ARCHIVES"])}_{row["rico:location"]}_{row["rico:isOrWasDigitalInstantiationOf"]}')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))
#write the graph to a file
g.serialize(destination=f'{OUTPUT_PATH}/sources.ttl', format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:94_Gentilly_3P255 a rico:Instantiation ;
    rico:hasOrHadDerivedInstanciation source:94_Gentilly_FRAD094_3P_000255_01 ;
    rico:hasOrHadPhysicalLocation "Archives Départementales du Val-de-Marne (94)" ;
    rico:identifier "3P255" ;
    rico:isOrWasInstantiationOf source:94_Gentilly_MAT_B_NB_1813 .

source:94_Gentilly_3P256 a rico:Instantiation ;
    rico:hasOrHadDerivedInstanciation source:94_Gentilly_FRAD094_3P_000256_01 ;
    rico:hasOrHadPhysicalLocation "Archives Départementales du Val-de-Marne (94)" ;
    rico:identifier "3P256" ;
    rico:isOrWasInstantiationOf source:94_Gentilly_MAT

<Graph identifier=N9cc402b4c6fd4ce2b4eb19fda2b02c37 (<class 'rdflib.graph.Graph'>)>